# Solution: Interventions in an age-structured population

Loading the model code from the previous etivity:

In [1]:
# PACKAGES
require(deSolve)
require(reshape2)
require(ggplot2)

# INPUT

# Set up an empty contact matrix with rows for each age group and columns for each age group
contact_matrix <- matrix(0,nrow=3,ncol=3)
# Fill in the contract matrix
contact_matrix[1,1] = 7     # daily number of contacts that children make with each other
contact_matrix[1,2] = 5     # daily number of contacts that children make with adults
contact_matrix[1,3] = 1     # daily number of contacts that children make with the elderly
contact_matrix[2,1] = 2     # daily number of contacts that adults make with children
contact_matrix[2,2] = 9     # daily number of contacts that adults make with each other
contact_matrix[2,3] = 1     # daily number of contacts that adults make with the elderly
contact_matrix[3,1] = 1     # daily number of contacts that elderly people make with children
contact_matrix[3,2] = 3     # daily number of contacts that elderly people make with adults
contact_matrix[3,3] = 2     # daily number of contacts that elderly people make with each other
# The contact_matrix now looks exactly like the one in the etivity instructions. We add this matrix as a parameter below.

# Parameters
parameters <- c(b = 0.05,     # the probability of infection per contact is 5%
                contact_matrix = contact_matrix,   # the age-specific average number of daily contacts (defined above)
                gamma = 1/5)  # the rate of recovery is 1/5 per day

# Run simulation for 3 months
times <- seq(from = 0, to = 90, by = 0.1)

# MODEL FUNCTION
sir_age_model <- function(time, state, parameters) {  
  
  with(as.list(parameters), {
    
    n_agegroups <- 3                                 # number of age groups
    S <- state[1:n_agegroups]                        # assign to S the first 3 numbers in the initial_state_values vector
    I <- state[(n_agegroups+1):(2*n_agegroups)]      # assign to I numbers 4 to 6 in the initial_state_values vector
    R <- state[(2*n_agegroups+1):(3*n_agegroups)]    # assign to R numbers 7 to 9 in the initial_state_values vector
      
    N <- S+I+R     # people in S, I and R are added separately by age group, so N is also a vector of length 3
    
    # Defining the force of infection
      
    # Force of infection acting on susceptible children
    lambda <- b * contact_matrix %*% as.matrix(I/N) 
    # %*% is used to multiply matrices in R
    # the lambda vector contains the forces of infection for children, adults and the elderly (length 3)

    # The differential equations
    # Rate of change in children:
    dS <- -lambda * S             
    dI <- lambda * S - gamma * I
    dR <- gamma * I
    
    # Output
    return(list(c(dS, dI, dR))) 
  })
}

Loading required package: deSolve
Loading required package: reshape2
Loading required package: ggplot2


## Modelling vaccination scenarios

### If you can only give the vaccine to one of the 3 age groups, which one would you prioritise to minimise the number of infections in the elderly? Would this also be the best strategy to reduce the overall number of infections in the population?

In [2]:
# 1) Vaccinating only children
# There are more doses of vaccine than children in the population, so the vaccine coverage will be 100% in children
# and 0% in the other groups as per the question

vacc_cov1 <- 1                  # vaccine coverage in children
vacc_cov2 <- 0                  # vaccine coverage in adults
vacc_cov3 <- 0                  # vaccine coverage in the elderly

vacc_eff3 <- 0.5                # vaccine efficacy in the elderly (100% in the other age groups)

# Effective vaccine coverage for each age group:
p1 <- vacc_cov1
p2 <- vacc_cov2
p3 <- vacc_cov3 * vacc_eff3

# Population size in total and for each age group:
N <- 1000000
N1 <- 0.2*N
N2 <- 0.65*N
N3 <- 0.15*N

# Fill in initial state values for a naive population based on effective vaccine coverage:
initial_state_values <- c(S1 = N1-p1*N1,
                          S2 = N2-p2*N2,  
                          S3 = N3-p3*N3,
                          I1 = 1,        # the outbreak starts with 1 infected person (can be of either age) 
                          I2 = 0,
                          I3 = 0,
                          R1 = p1*N1,
                          R2 = p2*N2,   
                          R3 = p3*N3)

# Run model output
output <- as.data.frame(ode(y = initial_state_values, 
                                       times = times, 
                                       func = sir_age_model,
                                       parms = parameters))

# Calculate cumulative incidence in each age group:
results1 <- data.frame(child_cum_inc = output$S1[1]-output$S1[nrow(output)],
                       adult_cum_inc = output$S2[1]-output$S2[nrow(output)], 
                       elderly_cum_inc =  output$S3[1]-output$S3[nrow(output)],
                       total_cum_inc = sum(output[1,c("S1", "S2", "S3")])-sum(output[nrow(output),c("S1", "S2", "S3")]))
print(results1)

  child_cum_inc adult_cum_inc elderly_cum_inc total_cum_inc
1             0      572797.7        92941.04      665738.7


Giving all available vaccine doses to children would prevent all infections in children, but still result in 92,941 infections in the elderly, and a total number of infections of 665,739 by the end of the epidemic.

Let's compare this with the output when giving all the vaccine doses to adults or the elderly instead:

In [3]:
# 2) Vaccinating only adults
# Vaccine coverage in adults = 250,000/650,000
# 0% in the other groups as per the question

vacc_cov1 <- 0                  # vaccine coverage in children
vacc_cov2 <- 0.38               # vaccine coverage in adults
vacc_cov3 <- 0                  # vaccine coverage in the elderly

vacc_eff3 <- 0.5                # vaccine efficacy in the elderly (100% in the other age groups)

# Effective vaccine coverage for each age group:
p1 <- vacc_cov1
p2 <- vacc_cov2
p3 <- vacc_cov3 * vacc_eff3

# Population size in total and for each age group:
N <- 1000000
N1 <- 0.2*N
N2 <- 0.65*N
N3 <- 0.15*N

# Fill in initial state values for a naive population based on effective vaccine coverage:
initial_state_values <- c(S1 = N1-p1*N1,
                          S2 = N2-p2*N2,  
                          S3 = N3-p3*N3,
                          I1 = 1,        # the outbreak starts with 1 infected person (can be of either age) 
                          I2 = 0,
                          I3 = 0,
                          R1 = p1*N1,
                          R2 = p2*N2,   
                          R3 = p3*N3)

# Run model output
output <- as.data.frame(ode(y = initial_state_values, 
                                       times = times, 
                                       func = sir_age_model,
                                       parms = parameters))

# Calculate cumulative incidence in each age group:
results2 <- data.frame(child_cum_inc = output$S1[1]-output$S1[nrow(output)],
                       adult_cum_inc = output$S2[1]-output$S2[nrow(output)], 
                       elderly_cum_inc =  output$S3[1]-output$S3[nrow(output)],
                       total_cum_inc = sum(output[1,c("S1", "S2", "S3")])-sum(output[nrow(output),c("S1", "S2", "S3")]))

# 3) Vaccinating only elderly people
# There are more doses of vaccine than elderly people in the population, so the vaccine coverage will be 100% 
# 0% in the other groups as per the question

vacc_cov1 <- 0                  # vaccine coverage in children
vacc_cov2 <- 0                  # vaccine coverage in adults
vacc_cov3 <- 1                  # vaccine coverage in the elderly

vacc_eff3 <- 0.5                # vaccine efficacy in the elderly (100% in the other age groups)

# Effective vaccine coverage for each age group:
p1 <- vacc_cov1
p2 <- vacc_cov2
p3 <- vacc_cov3 * vacc_eff3

# Population size in total and for each age group:
N <- 1000000
N1 <- 0.2*N
N2 <- 0.65*N
N3 <- 0.15*N

# Fill in initial state values for a naive population based on effective vaccine coverage:
initial_state_values <- c(S1 = N1-p1*N1,
                          S2 = N2-p2*N2,  
                          S3 = N3-p3*N3,
                          I1 = 1,        # the outbreak starts with 1 infected person (can be of either age) 
                          I2 = 0,
                          I3 = 0,
                          R1 = p1*N1,
                          R2 = p2*N2,   
                          R3 = p3*N3)

# Run model output
output <- as.data.frame(ode(y = initial_state_values, 
                                       times = times, 
                                       func = sir_age_model,
                                       parms = parameters))

# Calculate cumulative incidence in each age group:
results3 <- data.frame(child_cum_inc = output$S1[1]-output$S1[nrow(output)],
                       adult_cum_inc = output$S2[1]-output$S2[nrow(output)], 
                       elderly_cum_inc =  output$S3[1]-output$S3[nrow(output)],
                       total_cum_inc = sum(output[1,c("S1", "S2", "S3")])-sum(output[nrow(output),c("S1", "S2", "S3")]))

print(rbind(results1, results2, results3))

  child_cum_inc adult_cum_inc elderly_cum_inc total_cum_inc
1           0.0      572797.7        92941.04      665738.7
2      181263.6      332793.4        89262.62      603319.6
3      188970.3      603879.1        50022.62      842872.0


The cumulative incidence in the elderly is lowest if we only vaccinate everyone in the elderly age group (only 50,023 infections), despite the low vaccine efficacy in this age group! However, with this strategy we also get a substantially larger total number of infections than if vaccinating only children or only adults (842,872 infections vs. 665,739 and 603,320 respectively).

The worst strategy for the given question would be to only vaccinate children, since this neither minimises the number of infections in the elderly nor in total.

### If you distribute the vaccine doses among the 3 age groups in proportion to their population size, which group would benefit the most in terms of the percentage reduction in the cumulative incidence achieved with vaccination? Is the reduction in the total number on infections in the elderly what you would expect given the lower vaccine efficacy in this age group?

First, we need to calculate the baseline prevalence without vaccination, then the model the vaccine scenario:

In [4]:
# Baseline prevalence (no vaccine)

vacc_cov1 <- 0                  # vaccine coverage in children
vacc_cov2 <- 0                  # vaccine coverage in adults
vacc_cov3 <- 0                  # vaccine coverage in the elderly

vacc_eff3 <- 0.5                # vaccine efficacy in the elderly (100% in the other age groups)

# Effective vaccine coverage for each age group:
p1 <- vacc_cov1
p2 <- vacc_cov2
p3 <- vacc_cov3 * vacc_eff3

# Population size in total and for each age group:
N <- 1000000
N1 <- 0.2*N
N2 <- 0.65*N
N3 <- 0.15*N

# Fill in initial state values for a naive population based on effective vaccine coverage:
initial_state_values <- c(S1 = N1-p1*N1,
                          S2 = N2-p2*N2,  
                          S3 = N3-p3*N3,
                          I1 = 1,        # the outbreak starts with 1 infected person (can be of either age) 
                          I2 = 0,
                          I3 = 0,
                          R1 = p1*N1,
                          R2 = p2*N2,   
                          R3 = p3*N3)

# Run model output
output <- as.data.frame(ode(y = initial_state_values, 
                                       times = times, 
                                       func = sir_age_model,
                                       parms = parameters))

# Calculate cumulative incidence in each age group:
baseline_results <- data.frame(child_cum_inc = output$S1[1]-output$S1[nrow(output)],
                               adult_cum_inc = output$S2[1]-output$S2[nrow(output)], 
                               elderly_cum_inc =  output$S3[1]-output$S3[nrow(output)],
                               total_cum_inc = sum(output[1,c("S1", "S2", "S3")])-
                                                 sum(output[nrow(output),c("S1", "S2", "S3")]))

# Distributing vaccine doses among age groups proportionally to population size
# 250,000 doses/1 million = 25% coverage in each age group

vacc_cov1 <- 0.25                  # vaccine coverage in children
vacc_cov2 <- 0.25                  # vaccine coverage in adults
vacc_cov3 <- 0.25                  # vaccine coverage in the elderly

vacc_eff3 <- 0.5                # vaccine efficacy in the elderly (100% in the other age groups)

# Effective vaccine coverage for each age group:
p1 <- vacc_cov1
p2 <- vacc_cov2
p3 <- vacc_cov3 * vacc_eff3

# Population size in total and for each age group:
N <- 1000000
N1 <- 0.2*N
N2 <- 0.65*N
N3 <- 0.15*N

# Fill in initial state values for a naive population based on effective vaccine coverage:
initial_state_values <- c(S1 = N1-p1*N1,
                          S2 = N2-p2*N2,  
                          S3 = N3-p3*N3,
                          I1 = 1,        # the outbreak starts with 1 infected person (can be of either age) 
                          I2 = 0,
                          I3 = 0,
                          R1 = p1*N1,
                          R2 = p2*N2,   
                          R3 = p3*N3)

# Run model output
output <- as.data.frame(ode(y = initial_state_values, 
                                       times = times, 
                                       func = sir_age_model,
                                       parms = parameters))

# Calculate cumulative incidence in each age group:
results4 <- data.frame(child_cum_inc = output$S1[1]-output$S1[nrow(output)],
                       adult_cum_inc = output$S2[1]-output$S2[nrow(output)], 
                       elderly_cum_inc =  output$S3[1]-output$S3[nrow(output)],
                       total_cum_inc = sum(output[1,c("S1", "S2", "S3")])-sum(output[nrow(output),c("S1", "S2", "S3")]))

print(baseline_results)
print(results4)

  child_cum_inc adult_cum_inc elderly_cum_inc total_cum_inc
1      190215.4      609095.5        109311.1      908621.9
  child_cum_inc adult_cum_inc elderly_cum_inc total_cum_inc
1        130785      412563.8        77407.67      620756.4


In [5]:
# Reduction in prevalence achieved with vaccination:
(baseline_results-results4)/baseline_results

child_cum_inc,adult_cum_inc,elderly_cum_inc,total_cum_inc
0.3124374,0.3226615,0.2918589,0.3168155


Actually, the percentage reduction in prevalence achieved with this vaccination strategy is very similar across the 3 age groups! It is slightly higher in children and adults than in the elderly; the vaccine reduces the cumulative incidence in children and adults by 31-32%, compared to a 29% reduction in the elderly. 

At first glance, it might seem counterintuitive that the reduction in incidence in the elderly is nearly as high as for children and adults, despite the vaccine efficacy and therefore the effective vaccine coverage being only half that of the other age groups. However, it makes sense when looking at the contact matrix:

In [6]:
print(contact_matrix)

     [,1] [,2] [,3]
[1,]    7    5    1
[2,]    2    9    1
[3,]    1    3    2


On average, elderly people in this population make more contacts with children and adults (1+3) than with other elderly people (2) per day, which is why they benefit from a lower proportion of infected children and adults achieved with vaccination as well. 